In [1]:
import numpy as np
import warnings
import sys
import pandas as pd
import scipy
from scipy.io import arff
if not sys.warnoptions:
    warnings.simplefilter("ignore")
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
data, meta = scipy.io.arff.loadarff('../mtr-datasets/atp7d.arff')
data = pd.DataFrame(data)
data.head()

,"""'departwb'""","""'departdaytype'""","""'fetchdays'""","""'fetchdow'""","""'dtd'""","""'ALLminpA'""","""'ALLminpA'l1""","""'ALLminpA'l2""","""'ALLminpA'l7""","""'ALLavgpA'""",...,"""fdisoweekday4""","""fdisoweekday5""","""fdisoweekday6""","""fdisoweekday7""",LBL+ALLminpA+bt7d_000,LBL+ALLminp0+bt7d_000,LBL+aDLminpA+bt7d_000,LBL+aCOminpA+bt7d_000,LBL+aFLminpA+bt7d_000,LBL+aUAminpA+bt7d_000
0,72.0,1.0,60.0,3.0,12.0,359.0,372.0,351.0,317.0,615.96,...,0.0,0.0,0.0,0.0,326.0,675.0,626.0,545.0,326.0,545.0
1,79.0,1.0,69.0,5.0,10.0,336.0,298.0,298.0,320.0,616.73,...,0.0,1.0,0.0,0.0,315.0,675.0,631.0,587.0,315.0,587.0
2,79.0,1.0,70.0,6.0,9.0,336.0,336.0,298.0,320.0,628.87,...,0.0,0.0,1.0,0.0,315.0,675.0,631.0,587.0,315.0,587.0
3,79.0,1.0,71.0,7.0,8.0,336.0,336.0,336.0,330.0,632.73,...,0.0,0.0,0.0,1.0,315.0,675.0,631.0,587.0,315.0,587.0
4,79.0,1.0,72.0,1.0,7.0,336.0,336.0,336.0,330.0,635.47,...,0.0,0.0,0.0,0.0,315.0,1182.0,720.0,720.0,315.0,720.0


In [3]:
X = data.iloc[:,:-6].values
y = data.iloc[:,-6:].values
#Making Scaler
x_scaler = StandardScaler()
y_scaler = StandardScaler()

X = x_scaler.fit_transform(X)
y = y_scaler.fit_transform(y)

print(X.shape,y.shape)

#Train-Test Split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=100)

(296, 411) (296, 6)


In [4]:
print("Train_X: ",X_train.shape)
print("Train_Y: ",y_train.shape)
print("Test_X: ",X_test.shape)
print("Test_Y: ",y_test.shape)

Train_X:  (207, 411)
Train_Y:  (207, 6)
Test_X:  (89, 411)
Test_Y:  (89, 6)


In [5]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,f1_score,r2_score
def evaluate(y_train,y_test):
    print("RMSE: ", mean_squared_error(y_train,y_test)**0.5)
    print("MAE: ", mean_absolute_error(y_train,y_test))
    print("R_Squared: ", r2_score(y_train,y_test))
    
    r2 = r2_score(y_train,y_test)
    
    n = 296
    p = 6

    adj_r2 = 1-(1-r2)*(n-1)/(n-p-1)
    print('Adjusted R-squared - ',adj_r2)

## Bidirectional Model


In [6]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout
#from keras.backend.tensorflow_backend import clear_session
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [7]:
ENS_COUNT = 2
#clear_session()

In [8]:
def create_c1node(X_feed,y_now):
    '''
    C1 node Architecture:
    attribute:512:256:1 [saperate for each class]
    loss: Binary crossentropy
    '''
    model = Sequential()
    model.add(Dense(128,activation='relu',input_shape=(X_feed.shape[1],),kernel_initializer='glorot_uniform'))
    model.add(Dense(64,activation='relu',kernel_initializer='glorot_uniform'))
    model.add(Dense(y_now.shape[1],activation='linear',kernel_initializer='glorot_uniform'))
    model.compile(loss='mse', optimizer='adam', metrics=['mse'])
    
    return model

## Forward Ensemble

In [9]:
ensembles_fwd = []
for j in range(ENS_COUNT//2):
    print("TRAINING ENSEMBLE {} :\n\n\n".format(j))
    #Model chain
    chain_forward = []

    #Training
    #x_t,_,y_t,_ = train_test_split(X_train,y_train,test_size=0.4)
    X_feed = X_train
    y_t = y_train
    for i in range(y_t.shape[1]):
        print("Training chain node ",i)
        y_now = y_t[:,[i,]].copy()
        print("Shapes:\n X = {} \n Y = {}".format(X_feed.shape,y_now.shape))

        node = create_c1node(X_feed,y_now)
        node.fit(X_feed,y_now,epochs=500,batch_size=50)
        print("Training of node {} complete\n\n".format(i))  
        #Checking accuracy of the node
        #---
        #Adding node to chain 
        chain_forward.append(node)
        #Updating X_feed
        X_feed = np.append(X_feed,y_now,axis=1)
    ensembles_fwd.append(chain_forward)

TRAINING ENSEMBLE 0 :



Training chain node  0
Shapes:
 X = (207, 411) 
 Y = (207, 1)





Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor

Epoch 1/500
207/207 [==============================] - 0s - loss: 0.8983 - mean_squared_error: 0.8983     
Epoch 2/500
207/207 [==============================] - 0s - loss: 0.3733 - mean_squared_error: 0.3733     
Epoch 3/500
207/207 [==============================] - 0s - loss: 0.6380 - mean_squared_error: 0.6380     
Epoch 4/500
207/207 [==============================] - 0s - loss: 0.3932 - mean_squared_error: 0.3932     
Epoch 5/500
207/207 [==============================] - 0s - loss: 0.2297 - mean_squared_error: 0.2297     
Epoch 6/500
207/207 [==============================] - 0s - loss: 0.1644 - mean_squared_error: 0.1644     
Epoch 7/500
207/207 [==============================] - 0s - loss: 0.1535 - mean_squared_error: 0.1535     
Epoch 8/500
207/207 [===================

207/207 [==============================] - 0s - loss: 0.0138 - mean_squared_error: 0.0138     
Epoch 63/500
207/207 [==============================] - 0s - loss: 0.0288 - mean_squared_error: 0.0288     
Epoch 64/500
207/207 [==============================] - 0s - loss: 0.0378 - mean_squared_error: 0.0378     
Epoch 65/500
207/207 [==============================] - 0s - loss: 0.0793 - mean_squared_error: 0.0793     
Epoch 66/500
207/207 [==============================] - 0s - loss: 0.0918 - mean_squared_error: 0.0918     
Epoch 67/500
207/207 [==============================] - 0s - loss: 0.0394 - mean_squared_error: 0.0394     
Epoch 68/500
207/207 [==============================] - 0s - loss: 0.0413 - mean_squared_error: 0.0413     
Epoch 69/500
207/207 [==============================] - 0s - loss: 0.0363 - mean_squared_error: 0.0363     
Epoch 70/500
207/207 [==============================] - 0s - loss: 0.1541 - mean_squared_error: 0.1541     
Epoch 71/500
207/207 [===================

207/207 [==============================] - 0s - loss: 1.3522e-05 - mean_squared_error: 1.3522e-05     
Epoch 135/500
207/207 [==============================] - 0s - loss: 1.4400e-05 - mean_squared_error: 1.4400e-05     
Epoch 136/500
207/207 [==============================] - 0s - loss: 1.2768e-05 - mean_squared_error: 1.2768e-05     
Epoch 137/500
207/207 [==============================] - 0s - loss: 1.1788e-05 - mean_squared_error: 1.1788e-05     
Epoch 138/500
207/207 [==============================] - 0s - loss: 8.0391e-06 - mean_squared_error: 8.0391e-06     
Epoch 139/500
207/207 [==============================] - 0s - loss: 8.5103e-06 - mean_squared_error: 8.5103e-06     
Epoch 140/500
207/207 [==============================] - 0s - loss: 1.4007e-05 - mean_squared_error: 1.4007e-05     
Epoch 141/500
207/207 [==============================] - 0s - loss: 1.2620e-05 - mean_squared_error: 1.2620e-05     
Epoch 142/500
207/207 [==============================] - 0s - loss: 1.1626e-05

207/207 [==============================] - 0s - loss: 1.1058e-05 - mean_squared_error: 1.1058e-05     
Epoch 204/500
207/207 [==============================] - 0s - loss: 1.0641e-05 - mean_squared_error: 1.0641e-05     
Epoch 205/500
207/207 [==============================] - 0s - loss: 6.9340e-06 - mean_squared_error: 6.9340e-06     
Epoch 206/500
207/207 [==============================] - 0s - loss: 6.9044e-06 - mean_squared_error: 6.9044e-06     
Epoch 207/500
207/207 [==============================] - 0s - loss: 7.7400e-06 - mean_squared_error: 7.7400e-06     
Epoch 208/500
207/207 [==============================] - 0s - loss: 1.1145e-05 - mean_squared_error: 1.1145e-05     
Epoch 209/500
207/207 [==============================] - 0s - loss: 9.3461e-06 - mean_squared_error: 9.3461e-06     
Epoch 210/500
207/207 [==============================] - 0s - loss: 9.6836e-06 - mean_squared_error: 9.6836e-06     
Epoch 211/500
207/207 [==============================] - 0s - loss: 1.4085e-05

207/207 [==============================] - 0s - loss: 0.0066 - mean_squared_error: 0.0066     
Epoch 274/500
207/207 [==============================] - 0s - loss: 0.0053 - mean_squared_error: 0.0053     
Epoch 275/500
207/207 [==============================] - 0s - loss: 0.0034 - mean_squared_error: 0.0034     
Epoch 276/500
207/207 [==============================] - 0s - loss: 0.0032 - mean_squared_error: 0.0032     
Epoch 277/500
207/207 [==============================] - 0s - loss: 0.0030 - mean_squared_error: 0.0030     
Epoch 278/500
207/207 [==============================] - 0s - loss: 0.0051 - mean_squared_error: 0.0051     
Epoch 279/500
207/207 [==============================] - 0s - loss: 0.0088 - mean_squared_error: 0.0088     
Epoch 280/500
207/207 [==============================] - 0s - loss: 0.0041 - mean_squared_error: 0.0041     
Epoch 281/500
207/207 [==============================] - 0s - loss: 0.0038 - mean_squared_error: 0.0038     
Epoch 282/500
207/207 [==========

...

207/207 [==============================] - 0s - loss: 7.9562e-04 - mean_squared_error: 7.9562e-04     
Epoch 217/500
207/207 [==============================] - 0s - loss: 6.3060e-04 - mean_squared_error: 6.3060e-04     
Epoch 218/500
207/207 [==============================] - 0s - loss: 0.0013 - mean_squared_error: 0.0013             
Epoch 219/500
207/207 [==============================] - 0s - loss: 0.0010 - mean_squared_error: 0.0010     
Epoch 220/500
207/207 [==============================] - 0s - loss: 6.1413e-04 - mean_squared_error: 6.1413e-04
Epoch 221/500
207/207 [==============================] - ETA: 0s - loss: 5.9742e-04 - mean_squared_error: 5.9742e- - 0s - loss: 5.3986e-04 - mean_squared_error: 5.3986e-04     
Epoch 222/500
207/207 [==============================] - 0s - loss: 9.1210e-04 - mean_squared_error: 9.1210e-04     
Epoch 223/500
207/207 [==============================] - 0s - loss: 8.8489e-04 - mean_squared_error: 8.8489e-04     
Epoch 224/500
207/207 [========

207/207 [==============================] - 0s - loss: 8.4695e-04 - mean_squared_error: 8.4695e-04     
Epoch 286/500
207/207 [==============================] - 0s - loss: 0.0012 - mean_squared_error: 0.0012     
Epoch 287/500
207/207 [==============================] - 0s - loss: 0.0015 - mean_squared_error: 0.0015             
Epoch 288/500
207/207 [==============================] - 0s - loss: 0.0010 - mean_squared_error: 0.0010             
Epoch 289/500
207/207 [==============================] - 0s - loss: 0.0010 - mean_squared_error: 0.0010     
Epoch 290/500
207/207 [==============================] - 0s - loss: 0.0019 - mean_squared_error: 0.0019     
Epoch 291/500
207/207 [==============================] - 0s - loss: 0.0018 - mean_squared_error: 0.0018     
Epoch 292/500
207/207 [==============================] - 0s - loss: 0.0016 - mean_squared_error: 0.0016     
Epoch 293/500
207/207 [==============================] - 0s - loss: 0.0013 - mean_squared_error: 0.0013     
Epoch 294

207/207 [==============================] - 0s - loss: 4.6384e-05 - mean_squared_error: 4.6384e-05     
Epoch 358/500
207/207 [==============================] - 0s - loss: 2.9357e-05 - mean_squared_error: 2.9357e-05     
Epoch 359/500
207/207 [==============================] - 0s - loss: 5.8350e-05 - mean_squared_error: 5.8350e-05     
Epoch 360/500
207/207 [==============================] - 0s - loss: 4.6360e-05 - mean_squared_error: 4.6360e-05     
Epoch 361/500
207/207 [==============================] - 0s - loss: 3.0306e-05 - mean_squared_error: 3.0306e-05     
Epoch 362/500
207/207 [==============================] - 0s - loss: 4.8857e-05 - mean_squared_error: 4.8857e-05     
Epoch 363/500
207/207 [==============================] - 0s - loss: 4.3999e-05 - mean_squared_error: 4.3999e-05     
Epoch 364/500
207/207 [==============================] - 0s - loss: 2.9415e-05 - mean_squared_error: 2.9415e-05     
Epoch 365/500
207/207 [==============================] - 0s - loss: 1.9809e-05

207/207 [==============================] - 0s - loss: 0.0097 - mean_squared_error: 0.0097     
Epoch 428/500
207/207 [==============================] - 0s - loss: 0.0077 - mean_squared_error: 0.0077     
Epoch 429/500
207/207 [==============================] - 0s - loss: 0.0064 - mean_squared_error: 0.0064     
Epoch 430/500
207/207 [==============================] - 0s - loss: 0.0098 - mean_squared_error: 0.0098     
Epoch 431/500
207/207 [==============================] - 0s - loss: 0.0073 - mean_squared_error: 0.0073     
Epoch 432/500
207/207 [==============================] - 0s - loss: 0.0076 - mean_squared_error: 0.0076     
Epoch 433/500
207/207 [==============================] - 0s - loss: 0.0070 - mean_squared_error: 0.0070     
Epoch 434/500
207/207 [==============================] - 0s - loss: 0.0093 - mean_squared_error: 0.0093     
Epoch 435/500
207/207 [==============================] - 0s - loss: 0.0081 - mean_squared_error: 0.0081     
Epoch 436/500
207/207 [==========

207/207 [==============================] - 0s - loss: 2.5459e-04 - mean_squared_error: 2.5459e-04     
Training of node 5 complete




## Backward  Ensemble

In [10]:
ensembles_rev = []
for j in range(ENS_COUNT//2):
    print("TRAINING ENSEMBLE {} :\n\n\n".format(j))
    #Model chain
    chain_reverse = []
    #Training
    #x_t,_,y_t,_ = train_test_split(X_train,y_train,test_size=0.4)
    X_feed = X_train
    y_t = y_train
    for i in range(1,y_t.shape[1]+1):
        print("Training chain node ",i)
        y_now = y_t[:,[-i,]].copy()
        print("Shapes:\n X = {} \n Y = {}".format(X_feed.shape,y_now.shape))

        node = create_c1node(X_feed,y_now)
        node.fit(X_feed,y_now,epochs=500,batch_size=50)
        print("Training of node {} complete\n\n".format(i))  
        #Checking accuracy of the node
        #---
        #Adding node to chain 
        chain_reverse.append(node)
        #Updating X_feed
        X_feed = np.append(X_feed,y_now,axis=1)
    ensembles_rev.append(chain_reverse)

TRAINING ENSEMBLE 0 :



Training chain node  1
Shapes:
 X = (207, 411) 
 Y = (207, 1)
Epoch 1/500
207/207 [==============================] - 0s - loss: 0.7378 - mean_squared_error: 0.7378     
Epoch 2/500
207/207 [==============================] - 0s - loss: 0.3959 - mean_squared_error: 0.3959     
Epoch 3/500
207/207 [==============================] - 0s - loss: 0.2406 - mean_squared_error: 0.2406     
Epoch 4/500
207/207 [==============================] - 0s - loss: 0.2051 - mean_squared_error: 0.2051     
Epoch 5/500
207/207 [==============================] - 0s - loss: 0.1470 - mean_squared_error: 0.1470     
Epoch 6/500
207/207 [==============================] - 0s - loss: 0.1509 - mean_squared_error: 0.1509     
Epoch 7/500
207/207 [==============================] - 0s - loss: 0.1673 - mean_squared_error: 0.1673     
Epoch 8/500
207/207 [==============================] - 0s - loss: 0.1847 - mean_squared_error: 0.1847     
Epoch 9/500
207/207 [==============================] - 0s

207/207 [==============================] - 0s - loss: 4.7998e-04 - mean_squared_error: 4.7998e-04     
Epoch 73/500
207/207 [==============================] - 0s - loss: 6.6163e-04 - mean_squared_error: 6.6163e-04     
Epoch 74/500
207/207 [==============================] - 0s - loss: 5.4402e-04 - mean_squared_error: 5.4402e-04     
Epoch 75/500
207/207 [==============================] - 0s - loss: 5.0405e-04 - mean_squared_error: 5.0405e-04     
Epoch 76/500
207/207 [==============================] - 0s - loss: 9.1412e-04 - mean_squared_error: 9.1412e-04     
Epoch 77/500
207/207 [==============================] - 0s - loss: 7.7149e-04 - mean_squared_error: 7.7149e-04     
Epoch 78/500
207/207 [==============================] - 0s - loss: 8.2606e-04 - mean_squared_error: 8.2606e-04     
Epoch 79/500
207/207 [==============================] - 0s - loss: 0.0010 - mean_squared_error: 0.0010             
Epoch 80/500
207/207 [==============================] - 0s - loss: 0.0023 - mean_squa

207/207 [==============================] - 0s - loss: 0.0136 - mean_squared_error: 0.0136     
Epoch 147/500
207/207 [==============================] - 0s - loss: 0.0158 - mean_squared_error: 0.0158     
Epoch 148/500
207/207 [==============================] - 0s - loss: 0.0135 - mean_squared_error: 0.0135     
Epoch 149/500
207/207 [==============================] - 0s - loss: 0.0098 - mean_squared_error: 0.0098     
Epoch 150/500
207/207 [==============================] - 0s - loss: 0.0097 - mean_squared_error: 0.0097     
Epoch 151/500
207/207 [==============================] - 0s - loss: 0.0071 - mean_squared_error: 0.0071     
Epoch 152/500
207/207 [==============================] - 0s - loss: 0.0055 - mean_squared_error: 0.0055     
Epoch 153/500
207/207 [==============================] - 0s - loss: 0.0046 - mean_squared_error: 0.0046     
Epoch 154/500
207/207 [==============================] - 0s - loss: 0.0045 - mean_squared_error: 0.0045             
Epoch 155/500
207/207 [==

207/207 [==============================] - 0s - loss: 0.0247 - mean_squared_error: 0.0247     
Epoch 218/500
207/207 [==============================] - 0s - loss: 0.0332 - mean_squared_error: 0.0332     
Epoch 219/500
207/207 [==============================] - 0s - loss: 0.0242 - mean_squared_error: 0.0242     
Epoch 220/500
207/207 [==============================] - 0s - loss: 0.0310 - mean_squared_error: 0.0310     
Epoch 221/500
207/207 [==============================] - 0s - loss: 0.0205 - mean_squared_error: 0.0205     
Epoch 222/500
207/207 [==============================] - 0s - loss: 0.0253 - mean_squared_error: 0.0253     
Epoch 223/500
207/207 [==============================] - 0s - loss: 0.0243 - mean_squared_error: 0.0243     
Epoch 224/500
207/207 [==============================] - 0s - loss: 0.0217 - mean_squared_error: 0.0217     
Epoch 225/500
207/207 [==============================] - 0s - loss: 0.0116 - mean_squared_error: 0.0116     
Epoch 226/500
207/207 [==========

207/207 [==============================] - 0s - loss: 7.4810e-05 - mean_squared_error: 7.4810e-05     
Epoch 290/500
207/207 [==============================] - 0s - loss: 6.1513e-05 - mean_squared_error: 6.1513e-05     
Epoch 291/500
207/207 [==============================] - 0s - loss: 4.2135e-05 - mean_squared_error: 4.2135e-05     
Epoch 292/500
207/207 [==============================] - 0s - loss: 3.8207e-05 - mean_squared_error: 3.8207e-05     
Epoch 293/500
207/207 [==============================] - 0s - loss: 2.6792e-05 - mean_squared_error: 2.6792e-05     
Epoch 294/500
207/207 [==============================] - 0s - loss: 3.8328e-05 - mean_squared_error: 3.8328e-05     
Epoch 295/500
207/207 [==============================] - 0s - loss: 2.7037e-05 - mean_squared_error: 2.7037e-05     
Epoch 296/500
207/207 [==============================] - 0s - loss: 3.3438e-05 - mean_squared_error: 3.3438e-05     
Epoch 297/500
207/207 [==============================] - 0s - loss: 2.0947e-05

...

207/207 [==============================] - 0s - loss: 0.2295 - mean_squared_error: 0.2295     
Epoch 156/500
207/207 [==============================] - 0s - loss: 0.2066 - mean_squared_error: 0.2066     
Epoch 157/500
207/207 [==============================] - 0s - loss: 0.2516 - mean_squared_error: 0.2516     
Epoch 158/500
207/207 [==============================] - 0s - loss: 0.1861 - mean_squared_error: 0.1861     
Epoch 159/500
207/207 [==============================] - 0s - loss: 0.1389 - mean_squared_error: 0.1389     
Epoch 160/500
207/207 [==============================] - 0s - loss: 0.0939 - mean_squared_error: 0.0939     
Epoch 161/500
207/207 [==============================] - 0s - loss: 0.0663 - mean_squared_error: 0.0663     
Epoch 162/500
207/207 [==============================] - 0s - loss: 0.0627 - mean_squared_error: 0.0627     
Epoch 163/500
207/207 [==============================] - 0s - loss: 0.0442 - mean_squared_error: 0.0442     
Epoch 164/500
207/207 [==========

207/207 [==============================] - 0s - loss: 1.1863e-05 - mean_squared_error: 1.1863e-05     
Epoch 228/500
207/207 [==============================] - 0s - loss: 5.7063e-06 - mean_squared_error: 5.7063e-06     
Epoch 229/500
207/207 [==============================] - 0s - loss: 5.7308e-06 - mean_squared_error: 5.7308e-06     
Epoch 230/500
207/207 [==============================] - 0s - loss: 6.9372e-06 - mean_squared_error: 6.9372e-06     
Epoch 231/500
207/207 [==============================] - 0s - loss: 1.1724e-05 - mean_squared_error: 1.1724e-05     
Epoch 232/500
207/207 [==============================] - 0s - loss: 1.0280e-05 - mean_squared_error: 1.0280e-05     
Epoch 233/500
207/207 [==============================] - 0s - loss: 8.7539e-06 - mean_squared_error: 8.7539e-06     
Epoch 234/500
207/207 [==============================] - 0s - loss: 7.3162e-06 - mean_squared_error: 7.3162e-06     
Epoch 235/500
207/207 [==============================] - 0s - loss: 8.0923e-06

207/207 [==============================] - 0s - loss: 1.4776e-06 - mean_squared_error: 1.4776e-06     
Epoch 298/500
207/207 [==============================] - 0s - loss: 1.9311e-06 - mean_squared_error: 1.9311e-06     
Epoch 299/500
207/207 [==============================] - 0s - loss: 1.7419e-06 - mean_squared_error: 1.7419e-06     
Epoch 300/500
207/207 [==============================] - 0s - loss: 2.2131e-06 - mean_squared_error: 2.2131e-06     
Epoch 301/500
207/207 [==============================] - 0s - loss: 2.4075e-06 - mean_squared_error: 2.4075e-06     
Epoch 302/500
207/207 [==============================] - 0s - loss: 1.7032e-06 - mean_squared_error: 1.7032e-06     
Epoch 303/500
207/207 [==============================] - 0s - loss: 1.5752e-06 - mean_squared_error: 1.5752e-06     
Epoch 304/500
207/207 [==============================] - 0s - loss: 2.5235e-06 - mean_squared_error: 2.5235e-06     
Epoch 305/500
207/207 [==============================] - 0s - loss: 1.6862e-06

207/207 [==============================] - 0s - loss: 8.4576e-07 - mean_squared_error: 8.4576e-07     
Epoch 368/500
207/207 [==============================] - 0s - loss: 1.0774e-06 - mean_squared_error: 1.0774e-06     
Epoch 369/500
207/207 [==============================] - 0s - loss: 1.0920e-06 - mean_squared_error: 1.0920e-06     
Epoch 370/500
207/207 [==============================] - 0s - loss: 8.1934e-07 - mean_squared_error: 8.1934e-07     
Epoch 371/500
207/207 [==============================] - 0s - loss: 1.0232e-06 - mean_squared_error: 1.0232e-06     
Epoch 372/500
207/207 [==============================] - 0s - loss: 1.8041e-06 - mean_squared_error: 1.8041e-06     
Epoch 373/500
207/207 [==============================] - 0s - loss: 1.6269e-06 - mean_squared_error: 1.6269e-06     
Epoch 374/500
207/207 [==============================] - 0s - loss: 8.5689e-07 - mean_squared_error: 8.5689e-07     
Epoch 375/500
207/207 [==============================] - 0s - loss: 1.7591e-06

207/207 [==============================] - 0s - loss: 0.0152 - mean_squared_error: 0.0152     
Epoch 438/500
207/207 [==============================] - 0s - loss: 0.0145 - mean_squared_error: 0.0145     
Epoch 439/500
207/207 [==============================] - 0s - loss: 0.0128 - mean_squared_error: 0.0128     
Epoch 440/500
207/207 [==============================] - 0s - loss: 0.0412 - mean_squared_error: 0.0412     
Epoch 441/500
207/207 [==============================] - 0s - loss: 0.1107 - mean_squared_error: 0.1107     
Epoch 442/500
207/207 [==============================] - 0s - loss: 0.0598 - mean_squared_error: 0.0598     
Epoch 443/500
207/207 [==============================] - 0s - loss: 0.0424 - mean_squared_error: 0.0424     
Epoch 444/500
207/207 [==============================] - 0s - loss: 0.0772 - mean_squared_error: 0.0772     
Epoch 445/500
207/207 [==============================] - 0s - loss: 0.0798 - mean_squared_error: 0.0798     
Epoch 446/500
207/207 [==========

## Mean Aggregation

In [11]:
##Getting the output from the ensembles
y_full_fw = []
y_full_re = []

for k in range(ENS_COUNT//2):
    print("\n\nENSEMBLE {}:\n\n".format(k+1))
    X_feed_fw = X_test.copy()
    X_feed_re = X_test.copy()
    j = 0
    #Model chain
    y_pred_fw = []
    y_pred_re = []
    for i in range(len(chain_forward)):
        node1 = ensembles_fwd[k][i]
        node2 = ensembles_rev[k][i]
        print("Getting op from chain node ",j)
        node_name = "Node" + str(i)
        print("Shapes:\n X = {}".format(X_feed_fw.shape))
        output_fw = node1.predict(X_feed_fw)
        output_rev = node2.predict(X_feed_re)
        y_pred_fw.append(output_fw)
        y_pred_re.append(output_rev)
        X_feed_fw = np.append(X_feed_fw,output_fw,axis=1)
        X_feed_re = np.append(X_feed_re,output_rev,axis=1)
        j+=1
    y_full_fw.append(y_pred_fw)
    y_full_re.append(y_pred_re)
        #Updating X_feed
            



ENSEMBLE 1:


Getting op from chain node  0
Shapes:
 X = (89, 411)
Getting op from chain node  1
Shapes:
 X = (89, 412)
Getting op from chain node  2
Shapes:
 X = (89, 413)
Getting op from chain node  3
Shapes:
 X = (89, 414)
Getting op from chain node  4
Shapes:
 X = (89, 415)
Getting op from chain node  5
Shapes:
 X = (89, 416)


In [12]:
##Calculating the mean
cross_chain_group = []
for j in range(ENS_COUNT//2):
    prediction = []
    y_pred_fw = y_full_fw[j].copy()
    y_pred_re = y_full_re[j].copy()
    fw = np.array(y_pred_fw)[:,:,0].transpose().copy()
    re = np.array(y_pred_re)[:,:,0].transpose().copy()
    for i in range(np.array(y_pred_fw)[:,:,0].transpose().shape[0]):
        fw_i = fw[i,:].reshape(1,-1)[0]
        re_i = re[i,:].reshape(1,-1)[0][::-1] 
        final = (fw_i+re_i)/2
        prediction.append(final)
    cross_chain_group.append(np.array(prediction))

In [13]:
#Adding all results
total = None
for x in cross_chain_group:
    if total is None:
        total = x
    else:
        total = total + x
#Calculating the average
predictions = (total/(ENS_COUNT//2))

### Evaluating Mean Aggregation

In [14]:
evaluate(y_scaler.inverse_transform(y_test),
         y_scaler.inverse_transform(np.array(predictions)))

RMSE:  42.52497303535736
MAE:  25.4662834196055
R_Squared:  0.7404901081200639
Adjusted R-squared -  0.7351023594997192
